In [2]:
from sim_trifinger_k import TeddyBear, TriFingerEnv

import cvxpy as cp
import numpy as np
import torch
import grasp_utils
import grasp_opt
import matplotlib.pyplot as plt
from cvxpylayers.torch import CvxpyLayer
from nerf_shared import config_parser, utils

# Setup config (since we're not running from command line).
parser = config_parser.config_parser()

# Use the teddy_bear config, but change some filepaths.
args = parser.parse_args(args=[
    '--config', 'nerf_shared/logs/isaac_teddy4/config.txt',
    '--basedir', 'nerf_shared/logs',
    '--datadir', 'nerf_shared/data/isaac_teddy'])

# nerf_shared requires us to default to cuda tensors.
torch.set_default_tensor_type(torch.cuda.FloatTensor)

%load_ext autoreload
%autoreload 2
%matplotlib widget

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.10.1+cu102
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu102 as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu102/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


# Controlling fingers to reach pre-grasp points

In [5]:
env = TriFingerEnv(viewer=False, robot=True, Obj=TeddyBear)
env.robot.load_nerf_models(env.object.name)
env.robot.nerfs_loaded

## Sampling Grasps

In [3]:
# Load nerf models, params from checkpoint.
coarse_model, fine_model = utils.create_nerf_models(args)
optimizer = utils.get_optimizer(coarse_model, fine_model, args)
utils.load_checkpoint(coarse_model, fine_model, optimizer, args, checkpoint_index=-1)
# Override rendering bounds for fingers.
bds_dict = {'near': 0.05, 'far': 0.15}
renderer = utils.get_renderer(args, bds_dict)

Found ckpts ['../nerf_shared/logs/isaac_teddy4/050000.tar', '../nerf_shared/logs/isaac_teddy4/100000.tar', '../nerf_shared/logs/isaac_teddy4/150000.tar', '../nerf_shared/logs/isaac_teddy4/200000.tar']
Reloading from ../nerf_shared/logs/isaac_teddy4/200000.tar
Not ndc!
{'training': True, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_modules': OrderedDict(), 'perturb': 1.0, 'N_importance': 128, 'N_samples': 64, 'use_viewdirs': True, 'white_bkgd': False, 'raw_noise_std': 0.0, 'ndc': False, 'lindisp': False, 'near': 0.05, 'far': 0.15}


## CVXPYLayers Example
```python
n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
A_tch = torch.randn(m, n, requires_grad=True)
b_tch = torch.randn(m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)
solution
```

In [19]:
env.step_gym()

In [12]:
env.robot

In [ ]:
initial-np.array([[0.09,0.09,-0.025], [0, -0.125, 0], [-0.09, 0.09, 0.025]]) 

In [23]:
env.robot.rb_states[[6, 11, 16], :3]

tensor([[ 0.1099,  0.0437,  0.2733],
        [-0.0171, -0.1170,  0.2733],
        [-0.0928,  0.0733,  0.2733]], device='cpu')

In [19]:
# Form grasp vars
bear_centroid = np.array([-0.0001444 ,  0.00412231,  0.08663063])
gp = np.array([[0.09, 0.09, 0.013], [-0.09, 0.09, 0.013], [0, -0.125, 0.013]]) + bear_centroid
grasp_points = torch.tensor(gp).reshape(1, 3, 3)
grasp_dirs = torch.tensor(bear_centroid) - grasp_points
grasp_vars = torch.cat([grasp_points, grasp_dirs], dim=-1)

# Sample grasp points
grasp_points, grad_ests, grasp_mask = grasp_utils.sample_grasps(grasp_vars, 10, coarse_model, fine_model, renderer)
n_f = 3 
grasp_points = grasp_points.reshape(-1, n_f, 3)
normals = grad_ests.reshape(-1, n_f, 3)
G = grasp_opt.grasp_matrix(grasp_points, normals)

grasp_opt.min_norm_vector_in_facet(G[0].cpu().numpy())

(0.010920122576211541,
 array([[2.07228986e-08],
        [1.84369331e-05],
        [5.41491119e-01],
        [3.28062766e-08],
        [4.11146969e-08],
        [5.94580129e-08],
        [2.07475929e-08],
        [4.58453983e-01],
        [3.62867745e-05]]))

In [33]:
G_ = G[0]

gsq = torch.linalg.cholesky(G_.T @ G_ + 1e-5 * torch.eye(9))
grasp_opt.min_norm_vector_in_facet(G_.cpu().numpy()), cvxpylayer(gsq)[0]

((0.010920122576211541,
  array([[2.07228986e-08],
         [1.84369331e-05],
         [5.41491119e-01],
         [3.28062766e-08],
         [4.11146969e-08],
         [5.94580129e-08],
         [2.07475929e-08],
         [4.58453983e-01],
         [3.62867745e-05]])),
 tensor([[-2.1892e-05],
         [-2.8563e-05],
         [ 8.3614e-04],
         [ 5.1693e-02],
         [ 1.6805e-02],
         [ 2.1187e-03],
         [ 3.3899e-01],
         [ 3.2742e-01],
         [ 2.6219e-01]]))

In [57]:
from pyhull import convex_hull as cvh

hull = cvh.ConvexHull(G[0].cpu().numpy())

len(hull.vertices)

27

In [83]:
verts = []
for i in range(10):
    hull = cvh.ConvexHull(G[i].cpu().numpy())
    verts.append(hull.vertices[:])

verts = set.union(*[{''.join([str(x) for x in v])
             for v in vs}
            for vs in verts])

In [129]:
%%time
costs = []
for G_ in G:
    gsq = torch.linalg.cholesky(G_.T @ G_ + 1e-5 * torch.eye(9))
    x = cvxpylayer(gsq)[0]
    cost = torch.sum(torch.square(gsq @ x))
    # cost = prob.value
    costs.append(cost)

CPU times: user 46.9 ms, sys: 10.1 ms, total: 57 ms
Wall time: 44.7 ms


In [133]:
%%time
costs2 = []
for G_ in G:
    cost, _ = grasp_opt.min_norm_vector_in_facet(G_.cpu().numpy())
    costs2.append(cost)

CPU times: user 29.3 ms, sys: 40.7 ms, total: 70 ms
Wall time: 33.1 ms


In [134]:
costs

[tensor(4.2305e-06),
 tensor(4.7090e-06),
 tensor(5.0606e-06),
 tensor(5.9471e-06),
 tensor(5.0698e-06),
 tensor(6.7178e-06),
 tensor(2.0134e-06),
 tensor(1.3947e-05),
 tensor(3.5483e-06),
 tensor(2.1862e-06)]

In [136]:
plt.plot([x.cpu().numpy() for x in costs])

In [ ]:
costs2

In [79]:
len(set.union(v0, v0))

27

In [82]:
sum([len(v) for v in verts])

256

In [21]:
n_f = 3
n = n_f * 3  # OR 6 in the case of 
P = cp.Parameter((n, n), value=gsq.cpu().numpy())
# P = P.T @ P + 1e-10 * np.eye(n)
F = -np.eye(n)
# h = G @ np.random.randn(n)
h = np.zeros((n, 1))
A = np.ones((1, n))
b = np.ones(1)
x = cp.Variable((n, 1))

quad_form = cp.sum_squares(P @ x)

prob = cp.Problem(cp.Minimize(quad_form),
                 [F @ x <= h,
                  A @ x == b])
cvxpylayer = CvxpyLayer(prob, parameters=[P], variables=[x])

prob.is_qp()